In [3]:
import psycopg2
from utils.db_utils import conn_engine
from utils.db_utils import exec_query
from utils.etl_utils import unix_ts_to_datetime
from utils.etl_utils import datetime_to_unix_ts

In [22]:
def get_last_time(table_name):
    last_time = exec_query(f"SELECT MAX(t) FROM {table_name}")
    last_time = unix_ts_to_datetime(last_time)
    return last_time
    
    
    
def get_current_time():
    current_time = unix_ts_to_datetime(time.time())
    return current_time
 
    
    

        

In [ ]:
def etl_daily(start, end):
    df_daily = get_daily_data(start, end)
    buffer = StringIO()
    df_daily.to_csv(buffer, 
                    index_label=['open_price','close_price','low_price','high_price','status',
                                 'time_stamp_unix','volume','time_stamp_nyc', 'security_symbol'], 
                    header=False,
                    index = False)
    buffer.seek(0)
    with conn_engine() as db:
        with db.cursor() as cursor:
            cursor.copy_from(buffer, table='us_equity_daily_stage', sep=",",
                             columns=('open_price','close_price','low_price','high_price','status',
                                      'time_stamp_unix','volume','time_stamp_nyc', 'security_symbol'))
        db.commit()
        
        
    

In [ ]:
def twilio_message(message):
    from twilio.rest import Client

    account = twilio_config['sid']
    token = twilio_config['token']
    client = Client(account, token)

    message = client.messages.create(to="+12269781006", from_="+15555555555",
                                     body=message)

In [1]:
from utils.etl_utils import get_intraday_data
from utils.etl_utils import get_daily_data

In [2]:
intra2021 = get_intraday_data('2021-01-01 00:00:00', '2022-01-01 00:00:00')

ReadTimeout: HTTPSConnectionPool(host='finnhub.io', port=443): Read timed out. (read timeout=10)

In [ ]:
intra2021.info()

In [9]:
daily2021 = get_daily_data('2020-12-01 00:00:00', '2021-12-31 00:00:00')

In [6]:
daily2021

,c,h,l,o,s,t,v,tt,symbol
0,2.43,2.4600,2.27,2.34,ok,1609718400,1258022,2021-01-04 00:00:00+00:00,OVID
1,2.49,2.5200,2.39,2.45,ok,1609804800,1131399,2021-01-05 00:00:00+00:00,OVID
2,2.42,2.5400,2.38,2.50,ok,1609891200,1614176,2021-01-06 00:00:00+00:00,OVID
3,2.50,2.5100,2.39,2.46,ok,1609977600,1162893,2021-01-07 00:00:00+00:00,OVID
4,2.42,2.5600,2.40,2.49,ok,1610064000,1093514,2021-01-08 00:00:00+00:00,OVID
...,...,...,...,...,...,...,...,...,...
49007,61.05,63.5400,60.15,63.00,ok,1640563200,1337426,2021-12-27 00:00:00+00:00,OSTK
49008,60.12,62.7300,59.70,61.19,ok,1640649600,1020508,2021-12-28 00:00:00+00:00,OSTK
49009,59.72,60.8100,57.50,60.00,ok,1640736000,1090213,2021-12-29 00:00:00+00:00,OSTK
49010,60.64,62.2355,58.94,58.94,ok,1640822400,955375,2021-12-30 00:00:00+00:00,OSTK


In [ ]:
import pickle
import os

In [ ]:
file = 'intra2021.pkl'

In [ ]:
with open(file, 'wb') as f:
    pickle.dump(intra2021, f)

In [10]:
file_daily = 'daily2021.pkl'
with open(file_daily, 'wb') as f:
    pickle.dump(daily2021, f)